In [3]:
import os, sys

project_root = os.path.abspath(os.path.join(os.getcwd(), os.pardir))
src_path     = os.path.join(project_root, "src")

sys.path.insert(0, src_path)

print(f"üîç Added to sys.path: {src_path}")


üîç Added to sys.path: /Users/fabricesashinkumar/Documents/GitHub/Simulator/src


In [4]:
# ‚îÄ‚îÄ‚îÄ 0) Make sure we can import your src/ code ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ
import os, sys
project_root = os.path.abspath(os.path.join(os.getcwd(), ".."))
src_path     = os.path.join(project_root, "src")
sys.path.insert(0, src_path)

# ‚îÄ‚îÄ‚îÄ 1) Imports ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from models.simulator         import simulate_paths
from validation.cumulants     import cumulants
from models.params            import param_assign
from models.monte_carlo       import MonteCarloEngine  

# ‚îÄ‚îÄ‚îÄ 2) Define model categories ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ
all_models   = ["BM","ABM","GBM","VG","NIG","CGMY","MJD","KJD","POI","GAMMA","CIR","HESTON","CEV","SABR"]
models       = [m for m in all_models if m!="CGMY"]
log_models   = {"GBM","VG","NIG","MJD","KJD","HESTON","SABR"}
level_models = {"POI","GAMMA","CIR","CEV"}

# ‚îÄ‚îÄ‚îÄ 3) Time grid and check indices ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ
T      = 1.0
nsteps = 252
nsim   = 100_000
dt     = T / nsteps
tj     = np.linspace(0, T, nsteps+1)
check_idxs = [int(0.25*nsteps), int(0.50*nsteps), int(0.75*nsteps), nsteps]

all_results = []

# ‚îÄ‚îÄ‚îÄ 4) Validation loop ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ
for model in models:
    print(f"Validating {model} ‚Ä¶", end=" ")
    params = param_assign(model)

    # wrap simulate_paths(model,‚Ä¶) into our engine
    engine = MonteCarloEngine(
        model_simulator = lambda p, nsteps, nsim, dt, seed=None, **kw: 
                              simulate_paths(model, nsteps, nsim, dt, seed=seed),
        params = params,
        nsteps = nsteps,
        nsim   = nsim,
        dt     = dt,
        seed   = 42
    )

    # generate paths
    paths = engine.simulate()   # DataFrame (nsteps+1)√ónsim

    # extract slices and compute increments/log‚Äëreturns
    X = paths.iloc[check_idxs].values
    if model in log_models:
        S0 = paths.iloc[0].values
        X  = np.log(X / S0)
    elif model in level_models:
        L0 = paths.iloc[0].values
        X  = X - L0

    EX_mc = X.mean(axis=1)
    VX_mc = X.var(axis=1)

    # analytic cumulants per unit‚Äëtime
    try:
        c1, c2, *_ = cumulants(params, model, dt, tj=tj)
    except Exception as e:
        print(f"‚ùå cumulants: {e}")
        continue

    EX_an = c1 * tj[check_idxs] if c1 is not None else np.full_like(EX_mc, np.nan)
    VX_an = c2 * tj[check_idxs] if c2 is not None else np.full_like(VX_mc, np.nan)

    for i, t in enumerate(tj[check_idxs]):
        all_results.append({
            "Model":        model,
            "Time":         t,
            "E_MC":         EX_mc[i],
            "E_Analytic":   EX_an[i],
            "Var_MC":       VX_mc[i],
            "Var_Analytic": VX_an[i],
            "Bias_E":       EX_mc[i] - EX_an[i],
            "Bias_V":       VX_mc[i] - VX_an[i],
        })

    print("‚úîÔ∏è")

# ‚îÄ‚îÄ‚îÄ 5) Assemble & display ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ
df = pd.DataFrame(all_results)
display(df)

# ‚îÄ‚îÄ‚îÄ 6) Plot percent biases ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ
df["Bias_E_%"] = 100 * df["Bias_E"] / (np.abs(df["E_Analytic"]) + 1e-12)
df["Bias_V_%"] = 100 * df["Bias_V"] / (np.abs(df["Var_Analytic"]) + 1e-12)

pivot_E = df.pivot(index="Time", columns="Model", values="Bias_E_%")
pivot_V = df.pivot(index="Time", columns="Model", values="Bias_V_%")

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))
pivot_E.plot(ax=ax1, title="Percent Bias in Mean")
pivot_V.plot(ax=ax2, title="Percent Bias in Variance")
plt.tight_layout()
plt.show()


Validating BM ‚Ä¶ ‚úîÔ∏è
Validating ABM ‚Ä¶ ‚úîÔ∏è
Validating GBM ‚Ä¶ ‚úîÔ∏è
Validating VG ‚Ä¶ ‚úîÔ∏è
Validating NIG ‚Ä¶ ‚úîÔ∏è
Validating MJD ‚Ä¶ ‚úîÔ∏è
Validating KJD ‚Ä¶ ‚úîÔ∏è
Validating POI ‚Ä¶ ‚úîÔ∏è
Validating GAMMA ‚Ä¶ ‚úîÔ∏è
Validating CIR ‚Ä¶ ‚úîÔ∏è
Validating HESTON ‚Ä¶ ‚úîÔ∏è
Validating CEV ‚Ä¶ ‚úîÔ∏è
Validating SABR ‚Ä¶ ‚úîÔ∏è


Model  Time          E_MC  E_Analytic        Var_MC  Var_Analytic  \
0       BM  0.25 -9.247048e-04    0.000000  2.485488e-01  2.500000e-01   
1       BM  0.50 -1.558436e-03    0.000000  4.997437e-01  5.000000e-01   
2       BM  0.75  7.879324e-04    0.000000  7.468960e-01  7.500000e-01   
3       BM  1.00 -8.858523e-04    0.000000  9.973199e-01  1.000000e+00   
4      ABM  0.25 -9.247048e-04    0.000000  2.485488e-01  2.500000e-01   
5      ABM  0.50 -1.558436e-03    0.000000  4.997437e-01  5.000000e-01   
6      ABM  0.75  7.879324e-04    0.000000  7.468960e-01  7.500000e-01   
7      ABM  1.00 -8.858523e-04    0.000000  9.973199e-01  1.000000e+00   
8      GBM  0.25  7.315059e-03    0.007500  9.941950e-03  1.000000e-02   
9      GBM  0.50  1.468831e-02    0.015000  1.998975e-02  2.000000e-02   
10     GBM  0.75  2.265759e-02    0.022500  2.987584e-02  3.000000e-02   
11     GBM  1.00  2.982283e-02    0.030000  3.989280e-02  4.000000e-02   
12      VG  0.25  2.432385e-03    0.002500  1.004118e-02  1.003443e-02   
13      VG  0.50  4.374031e-03    0.005000  2.017593e-02  2.006885e-02   
14      VG  0.75  6.180069e-03    0.007500  3.036721e-02  3.010328e-02   
15      VG  1.00  8.187282e-03    0.010000  4.055141e-02  4.013770e-02   
16     NIG  0.25 -4.868935e-02   -0.008176  4.976404e-02  8.533022e-03   
17     NIG  0.50 -9.901294e-02   -0.016352  9.871712e-02  1.706604e-02   
18     NIG  0.75 -1.471552e-01   -0.024529  1.490817e-01  2.559907e-02   
19     NIG  1.00 -1.965189e-01   -0.032705  1.995091e-01  3.413209e-02   
20     MJD  0.25 -9.640365e-03   -0.009375  1.231805e-02  1.234375e-02   
21     MJD  0.50 -1.912560e-02   -0.018750  2.475399e-02  2.468750e-02   
22     MJD  0.75 -2.814051e-02   -0.028125  3.707874e-02  3.703125e-02   
23     MJD  1.00 -3.806086e-02   -0.037500  4.955658e-02  4.937500e-02   
24     KJD  0.25 -3.108382e-01   -0.312500  7.392281e+00  7.301667e+00   
25     KJD  0.50 -6.206591e-01   -0.625000  1.477955e+01  1.460333e+01   
26     KJD  0.75 -9.313412e-01   -0.937500  2.224501e+01  2.190500e+01   
27     KJD  1.00 -1.245806e+00   -1.250000  2.956703e+01  2.920667e+01   
28     POI  0.25  1.250140e+00    1.250000  1.257230e+00  1.250000e+00   
29     POI  0.50  2.501770e+00    2.500000  2.499697e+00  2.500000e+00   
30     POI  0.75  3.752350e+00    3.750000  3.737179e+00  3.750000e+00   
31     POI  1.00  5.003990e+00    5.000000  4.998114e+00  5.000000e+00   
32   GAMMA  0.25  1.246682e-01    0.125000  1.244471e-02  1.250000e-02   
33   GAMMA  0.50  2.497805e-01    0.250000  2.479746e-02  2.500000e-02   
34   GAMMA  0.75  3.741839e-01    0.375000  3.724793e-02  3.750000e-02   
35   GAMMA  1.00  4.995821e-01    0.500000  4.948107e-02  5.000000e-02   
36     CIR  0.25  1.151856e-02    0.011993  4.227977e-04  4.004754e-04   
37     CIR  0.50  2.223183e-02    0.023986  8.960085e-04  8.009509e-04   
38     CIR  0.75  3.227954e-02    0.035979  1.388564e-03  1.201426e-03   
39     CIR  1.00  4.142682e-02    0.047971  1.907977e-03  1.601902e-03   
40  HESTON  0.25 -5.721824e-03   -0.005221  1.142324e-02  1.641362e-03   
41  HESTON  0.50 -1.328339e-02   -0.010442  2.578638e-02  3.282723e-03   
42  HESTON  0.75 -2.183490e-02   -0.015664  4.311426e-02  4.924085e-03   
43  HESTON  1.00 -3.184206e-02   -0.020885  6.294275e-02  6.565446e-03   
44     CEV  0.25  2.499998e-02    0.025000  9.937067e-11  1.000000e-10   
45     CEV  0.50  4.999997e-02    0.050000  1.996991e-10  2.000000e-10   
46     CEV  0.75  7.500002e-02    0.075000  2.983134e-10  3.000000e-10   
47     CEV  1.00  9.999998e-02    0.100000  3.981343e-10  4.000000e-10   
48    SABR  0.25 -3.322830e-07         NaN  2.509986e-07           NaN   
49    SABR  0.50 -2.225080e-06         NaN  5.028323e-07           NaN   
50    SABR  0.75 -2.771846e-06         NaN  7.591232e-07           NaN   
51    SABR  1.00 -4.299791e-06         NaN  1.014834e-06           NaN   

          Bias_E        Bias_V  
0  -9.247048e-04 -1.451247e-03  
1  -1.558436e-03

NameError: name 'payoff' is not defined